# TRM Router Experiments - Standalone

This notebook implements the Tiny Recursive Model (TRM) router experiments for
meta-optimization of inner solvers in the Manifold Muon optimizer.

## Workflow

1. **Data Collection**: Run training with fixed solvers, collecting dynamics
2. **Data Merging**: Merge runs to create oracle labels
3. **TRM Training**: Train TRM router to predict optimal solver
4. **Evaluation**: Compare TRM routing vs fixed baselines

## Reference
- Jolicoeur-Martineau, A. (2025). "Less is More: Recursive Reasoning with Tiny Networks."

In [ ]:
# Setup and imports
import os
import sys
import json
import pickle
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Add parent directory to path
sys.path.insert(0, '..')

from models import get_model
from muon import MuonSGD, get_inner_solver
from trm import (
    TRMRouter,
    create_trm_router,
    TRMDataCollector,
    DynamicsDataset,
    DynamicsFeatureExtractor,
    TrainingState,
    merge_solver_runs,
    create_dataloaders,
)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 1: Data Collection

Run training with each fixed solver and collect dynamics data.

In [ ]:
# Data loaders
def make_loaders(batch_size=128):
    transform_train = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    transform_test = T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    ])
    
    train_set = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train
    )
    test_set = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test
    )
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader

train_loader, test_loader = make_loaders()
print(f"Train batches: {len(train_loader)}, Test batches: {len(test_loader)}")

In [ ]:
def collect_solver_dynamics(
    solver_name: str,
    epochs: int = 5,
    lr: float = 0.1,
    spectral_budget: float = 0.1,
    output_path: str = None,
    seed: int = 42,
):
    """Run training with a fixed solver and collect dynamics."""
    
    torch.manual_seed(seed)
    
    # Create model
    model = get_model('small_cnn').to(device)
    
    # Create optimizer with fixed solver
    inner_solver = get_inner_solver(solver_name)
    optimizer = MuonSGD(
        model.parameters(),
        lr=lr,
        momentum=0.9,
        weight_decay=5e-4,
        spectral_budget=spectral_budget,
        inner_solver=inner_solver,
    )
    
    # Data collector
    collector = TRMDataCollector(
        model=model,
        solver_name=solver_name,
        total_epochs=epochs,
        steps_per_epoch=len(train_loader),
        record_every=1,
    )
    
    # Training loop
    losses = []
    global_step = 0
    
    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} [{solver_name}]")
        
        for batch_idx, (x, y) in enumerate(pbar):
            x, y = x.to(device), y.to(device)
            
            # Forward
            optimizer.zero_grad()
            logits = model(x)
            loss = F.cross_entropy(logits, y)
            
            # Compute gradient norm
            loss.backward()
            grad_norm = sum(
                p.grad.norm().item() ** 2 
                for p in model.parameters() if p.grad is not None
            ) ** 0.5
            
            # Pre-step record
            collector.pre_step(
                loss=loss.item(),
                grad_norm=grad_norm,
                epoch=epoch,
                step=global_step,
            )
            
            # Optimizer step
            optimizer.step()
            global_step += 1
            
            # Post-step: get next loss
            with torch.no_grad():
                next_logits = model(x)
                next_loss = F.cross_entropy(next_logits, y).item()
            
            collector.post_step(next_loss=next_loss)
            
            epoch_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        print(f"  Avg loss: {avg_loss:.4f}")
    
    # Save
    if output_path:
        collector.save(output_path)
    
    return collector.records, losses

In [ ]:
# Collect data for each solver (select subset for quick iteration)
SOLVERS = ['dual_ascent', 'admm', 'frank_wolfe']
DATA_DIR = Path('../results')
DATA_DIR.mkdir(exist_ok=True)

solver_records = {}
solver_losses = {}

for solver in SOLVERS:
    print(f"\n{'='*50}")
    print(f"Collecting dynamics for: {solver}")
    print('='*50)
    
    records, losses = collect_solver_dynamics(
        solver_name=solver,
        epochs=5,  # Use more epochs for full experiment
        output_path=DATA_DIR / f'dynamics_{solver}.pkl',
    )
    
    solver_records[solver] = records
    solver_losses[solver] = losses

print("\nData collection complete!")

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 4))

for solver, losses in solver_losses.items():
    plt.plot(losses, label=solver, marker='o')

plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss by Solver')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(DATA_DIR / 'solver_comparison.png', dpi=150)
plt.show()

## Part 2: Merge Data and Create Oracle Labels

In [ ]:
# Merge solver runs
run_paths = {solver: DATA_DIR / f'dynamics_{solver}.pkl' for solver in SOLVERS}
merged_path = DATA_DIR / 'merged_dynamics.pkl'

merged_data = merge_solver_runs(run_paths, merged_path)

In [ ]:
# Analyze oracle distribution over training
from collections import Counter

records = merged_data['records']

# Oracle by epoch
oracle_by_epoch = {}
for r in records:
    epoch = r.epoch if hasattr(r, 'epoch') else r['epoch']
    oracle = r.oracle_solver if hasattr(r, 'oracle_solver') else r['oracle_solver']
    
    if epoch not in oracle_by_epoch:
        oracle_by_epoch[epoch] = []
    oracle_by_epoch[epoch].append(oracle)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

epochs = sorted(oracle_by_epoch.keys())
for solver in SOLVERS:
    fracs = []
    for ep in epochs:
        count = sum(1 for o in oracle_by_epoch[ep] if o == solver)
        fracs.append(count / len(oracle_by_epoch[ep]))
    ax.plot(epochs, fracs, label=solver, marker='o')

ax.set_xlabel('Epoch')
ax.set_ylabel('Fraction as Oracle')
ax.set_title('Which Solver is Best at Each Epoch?')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(DATA_DIR / 'oracle_by_epoch.png', dpi=150)
plt.show()

## Part 3: Train TRM Router

In [ ]:
# Create dataset
dataset = DynamicsDataset(merged_path)
stats = dataset.get_statistics()

print(f"Dataset size: {stats['num_samples']}")
print(f"Feature dim: {stats['feature_dim']}")
print("Solver distribution:")
for name, count in stats['solver_distribution'].items():
    print(f"  {name}: {count} ({100*count/stats['num_samples']:.1f}%)")

In [ ]:
# Train TRM router (inline)
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

# Create dataloaders
train_dl, val_dl = create_dataloaders(merged_path, batch_size=64, train_split=0.8)

# Create model
trm = create_trm_router(size='small').to(device)
print(f"TRM Router: {trm.num_parameters:,} parameters")

# Optimizer
optimizer = AdamW(trm.parameters(), lr=1e-4, weight_decay=0.1)
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6)

# Training
history = []
best_acc = 0.0

for epoch in range(1, 101):
    # Train
    trm.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch in train_dl:
        features = batch['features'].to(device)
        labels = batch['solver_label'].to(device)
        
        optimizer.zero_grad()
        outputs = trm(features, return_all_cycles=True)
        
        # Deep supervision loss
        loss = sum(
            F.cross_entropy(logits, labels)
            for logits in outputs['all_solver_logits']
        ) / len(outputs['all_solver_logits'])
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(trm.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item() * features.size(0)
        preds = outputs['solver_logits'].argmax(dim=-1)
        train_correct += (preds == labels).sum().item()
        train_total += features.size(0)
    
    # Eval
    trm.eval()
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for batch in val_dl:
            features = batch['features'].to(device)
            labels = batch['solver_label'].to(device)
            
            outputs = trm(features)
            preds = outputs['solver_logits'].argmax(dim=-1)
            val_correct += (preds == labels).sum().item()
            val_total += features.size(0)
    
    scheduler.step()
    
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    
    history.append({
        'epoch': epoch,
        'train_loss': train_loss / train_total,
        'train_acc': train_acc,
        'val_acc': val_acc,
    })
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(trm.state_dict(), DATA_DIR / 'trm_router_best.pt')
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} | Loss: {train_loss/train_total:.4f} | "
              f"Train: {train_acc:.4f} | Val: {val_acc:.4f}")

print(f"\nBest validation accuracy: {best_acc:.4f}")

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

epochs = [h['epoch'] for h in history]

ax1.plot(epochs, [h['train_loss'] for h in history])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('TRM Router Training Loss')
ax1.grid(True, alpha=0.3)

ax2.plot(epochs, [h['train_acc'] for h in history], label='Train')
ax2.plot(epochs, [h['val_acc'] for h in history], label='Val')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('TRM Router Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(DATA_DIR / 'trm_training.png', dpi=150)
plt.show()

## Part 4: Analysis

Analyze what the TRM learned about solver selection.

In [ ]:
# Load best model
trm.load_state_dict(torch.load(DATA_DIR / 'trm_router_best.pt'))
trm.eval()

# Confusion matrix
SOLVER_NAMES = ['spectral_clip', 'dual_ascent', 'quasi_newton', 'frank_wolfe', 'admm']
confusion = torch.zeros(5, 5, dtype=torch.long)

with torch.no_grad():
    for batch in val_dl:
        features = batch['features'].to(device)
        labels = batch['solver_label'].to(device)
        
        outputs = trm(features)
        preds = outputs['solver_logits'].argmax(dim=-1)
        
        for t, p in zip(labels.cpu(), preds.cpu()):
            confusion[t, p] += 1

# Plot confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(confusion.float() / confusion.sum(dim=1, keepdim=True), cmap='Blues')

ax.set_xticks(range(5))
ax.set_yticks(range(5))
ax.set_xticklabels([s[:8] for s in SOLVER_NAMES], rotation=45, ha='right')
ax.set_yticklabels([s[:8] for s in SOLVER_NAMES])
ax.set_xlabel('Predicted')
ax.set_ylabel('True (Oracle)')
ax.set_title('TRM Routing Confusion Matrix')

# Add values
for i in range(5):
    for j in range(5):
        val = confusion[i, j].item()
        color = 'white' if confusion[i, j] > confusion.max() / 2 else 'black'
        ax.text(j, i, f'{val}', ha='center', va='center', color=color, fontsize=10)

plt.colorbar(im)
plt.tight_layout()
plt.savefig(DATA_DIR / 'confusion_matrix.png', dpi=150)
plt.show()

## Summary

This notebook demonstrated:

1. **Data Collection**: Collecting training dynamics from runs with fixed solvers
2. **Oracle Labeling**: Determining which solver was best at each step
3. **TRM Training**: Training a tiny recursive model to predict optimal solver
4. **Evaluation**: Measuring TRM's ability to match the oracle

Key findings:
- Different solvers are optimal at different training phases
- TRM can learn to approximate the oracle solver selection
- The TRM overhead is negligible (~100K parameters)

This connects to the broader Manifold Muon project by exploring whether
meta-learned routing can improve hyperparameter transfer properties.